In [ ]:
%pip install --upgrade -r requirements.txt

Basic Anthropic Integration

In [ ]:
from anthropic import Anthropic 
import json
import os

#Read the API KEYs from the environment, replace the default values (the second argument) with your own keys if needed
anthropic_key = os.getenv("ANTHROPIC_API_KEY", "YOUR_ANTHROPIC_API_KEY")
payi_api_key = os.getenv("PAYI_API_KEY", "YOUR_PAYI_API_KEY")

payi_headers = {
    "xProxy-api-key": payi_api_key,
}

payi_base_url = "https://api.pay-i.com"
payi_anthropic_url = payi_base_url + "/api/v1/proxy/anthropic"

client = Anthropic(
    base_url=payi_anthropic_url,
    api_key=anthropic_key,
    default_headers=payi_headers
)

message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ]
)
completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))


Handle streaming calls. xproxy_result is returned as part of the last chunk.

In [ ]:
stream = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ],
    stream=True
)

message = ""
input_tokens = None
output_tokens = None
for event in stream:
    match event.type:
        case "message_start":
            input_tokens = event.message.usage.input_tokens
        case "content_block_start":
            message += event.content_block.text
        case "content_block_delta":
            message += event.delta.text
        case "message_delta":
            output_tokens = event.usage.output_tokens
            if 'xproxy_result' in event.model_extra:
                xproxy_result = event.model_extra['xproxy_result']
        case "content_block_stop" | "message_stop":
            ...

print(message)
if (xproxy_result is not None):
    print()
    print(json.dumps(xproxy_result, indent=4))

Use the Pay-i SDK to send a request with request tags

In [ ]:
from payi.lib.helpers import create_headers

message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a request tag test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(request_tags=["x", "y"])
)

completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))

Create a budget and make a request with that budget

In [ ]:
from payi import Payi

payi_client = Payi(
    api_key=payi_api_key
)

#Create a budget
budget_response = payi_client.budgets.create(
    #As long as the budget configuration remains the same across creates, the same budget name can be used repeatedly
    budget_name='Anthropic quickstart allow budget',
    max=12.50, #$12.50 USD
    budget_response_type="Allow",
    budget_tags=["example_budget"]
)

budget_name = budget_response.budget.budget_name
budget_id = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_name}")
print(f"Budget ID: {budget_id}")

#Make a request using the new budget and request tags
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(
        request_tags=["x", "y"],
        budget_ids=[budget_id]
    )
)

completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))

See budget status

In [ ]:
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.output.base}")

Make an ingest call with pre-computed token values

In [ ]:
response = payi_client.ingest.units(
    category="system.anthropic",
    resource="claude-3-sonnet-20240229",
    units={ "text": { "input": 50, "output": 100 } },
    budget_ids=[budget_id],
    request_tags=["a", "b"]
)

print(f"Ingest request ID: {response.request_id}")
print(f"Input Base Cost: {response.xproxy_result.cost.input.base}")
print(f"Output Base Cost: {response.xproxy_result.cost.output.base}")
print(f"Total Base Cost: {response.xproxy_result.cost.total.base}")

Reset a budget back to zero tracked cost

In [ ]:
response = payi_client.budgets.reset(budget_id=budget_id)
print(response.message)
print("State prior to reset: ")
print(f"Budget Name: {response.budget_history.budget_name}")
print(f"Budget ID: {response.budget_history.budget_id}")
print(f"Budget Tags: {response.budget_history.budget_tags}")
print(f"Budget Reset Timestamp: {response.budget_history.budget_reset_timestamp}")
print(f"Budget Input Base Cost: {response.budget_history.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget_history.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget_history.totals.cost.total.base}")

print("\nState after reset:")
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.total.base}")

Create a small, conservative budget that will prevent calls from happening that exceed the maximum, then capture the output.

In [ ]:
budget_response = payi_client.budgets.create(
    #As long as the budget configuration remains the same across creates, the same budget name can be used repeatedly
    budget_name='Anthropic quickstart block budget',
    max=0.00000001, 
    budget_response_type="block",
    budget_tags=["budget_block_example"]
)
block_budget = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_response.budget.budget_name}")
print(f"Budget ID: {budget_response.budget.budget_id}")

try:
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=500,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "provide me a list of toys for children 5 and under"
                    }
                ]
            }
        ],
        extra_headers=create_headers(
            request_tags=["x", "y"],
            budget_ids=[block_budget]
        )
    )

    completion = message.content[0].text
    print(completion)

    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=500,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "tell me a short story about a toy"
                    }
                ]
            }
        ],
        extra_headers=create_headers(
            request_tags=["x", "y"],
            budget_ids=[block_budget]
        )
    )
except Exception as e:
    print(json.dumps(e.body, indent=4))

Create an experience type and instance, then send a request with the experience instance

In [ ]:
# Create an experience type
exp_name="quickstart_experience"
exp_type_response = payi_client.experiences.types.create(
    name=exp_name,
    description="An example of an experience"
)

# Create an experience of the type
exp_response = payi_client.experiences.create(
    experience_name=exp_name,
)

# Make a request using the budget, request tags, and experience
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(
        request_tags=["x", "y"],
        budget_ids=[budget_id],
        experience_id=exp_response.experience_id
    )
)

completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))

Send a request with a budget and user ID

In [ ]:
# Make a request using the budget, request tags, and user id
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(
        budget_ids=[budget_id],
        # user id can be any string value
        user_id="example_user_id"
    )
)

completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))

List and then delete all budgets

In [ ]:
response = payi_client.budgets.list()
for budget in response.items:
    print("Deleting budget with id:" + budget.budget_id)
    payi_client.budgets.delete(budget.budget_id)